In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

## Optimization via Stochastic Gradient Descent

Given an integer batch_size, randomly extract a sub-dataset such that from the original dataset. Note that the random sampling at each iteration has to be done without replacement.

### functions

In [2]:
def GD(f, grad_f, x0, D, kmax,alpha):
    X, y = D
    N = X.shape[1]
    Xhat = np.concatenate((np.ones((1,N)), X), axis=0)
    xsol=x0
    x=[x0]
    i=0
    cond = True
    while i < kmax :
        x.append(xsol)
        i+=1
    return x
def sgd_optimizer(loss, grad_loss, w0, D, alpha, batch_size, n_epochs):
    X, y = D # Unpack the data\n",
    N = X.shape[1]
    d = w0.shape[0]
    idx = np.arange(0, N)
 
 
    # Initialization of history vectors\n",
    w_history = np.zeros((n_epochs, d)) # Save weights at each iteration\n",
    loss_history = np.zeros((n_epochs, )) # Save loss values at each iteration\n",
    grad_norm_history = np.zeros((n_epochs, )) # Save gradient norms at each iteration\n",

    # Initialize weights\n",
    w = w0 
    for epoch in range(n_epochs):
        # Shuffle the data at the beginning of each epoch
        np.random.shuffle(idx)
        X = X[:,idx]
        y = y[idx]

        # Initialize a vector that saves the gradient of the loss at each iteration\n",
        grad_loss_vec = []

        for batch_start in range(0, N, batch_size):
            batch_end = min(batch_start + batch_size, N)
            X_batch = X[:,batch_start:batch_end]
            y_batch = y[batch_start:batch_end]
            # Compute the gradient of the loss
            gradient = grad_loss(w, X_batch, y_batch)

            grad_loss_vec.append(np.linalg.norm(gradient, 2))

            # Update weights
            w = w - alpha* gradient

        # Save the updated values
        w_history[epoch] = w[:,0]
        loss_history[epoch] = loss(w, X, y)
        grad_norm_history[epoch] = np.mean(grad_loss_vec)

    return w_history, loss_history, grad_norm_history
 
 
 
def loss(f, w, D):
    X, y = D
    y_pred = f(w, X)
    return np.mean(np.square(y_pred -y))
def grad_l(f, grad_f, w, D):
    X, y = D
    return np.mean(2 *grad_f(w, X).T * (f(w, X) - y))

- Given an integer batch_size, randomly extract a sub-dataset 
 such that 
 from the original dataset. Note that the random sampling at each iteration has to be done without replacement.

In [5]:
data = loadmat('MNIST.mat')
X=data["X"]
Y=data["I"]
print(X.shape,Y.shape)

(256, 1707) (1, 1707)


In [6]:
C1=int(input())
C2=int(input())
idx=(Y[0,:]==C1 )|( Y[0,:]==C2)
X=X[:,idx]
Y=Y[:,idx]
for i in range(Y.shape[1]):
    if Y[:,i]==C1:
        Y[:,i]=0
    else:
        Y[:,i]=1
d,N=X.shape
Y = Y.reshape((N, 1))
print(X.shape)
print(Y.shape)

(256, 333)
(333, 1)


In [7]:
import numpy as np

def grad_loss_single(w, x, y):
    """
    Compute the gradient of the loss for a single sample (x, y).

    Example for a simple linear regression model:
    f(w, x) = w^T x
    Loss = (f(w, x) - y)^2
    grad Loss = 2 * (f(w, x) - y) * x

    Parameters:
    w: parameter vector, shape (d,)
    x: single input sample, shape (d,)
    y: scalar label

    Returns:
    grad: shape (d,), gradient of the loss w.r.t. w for this sample
    """
    # Compute prediction
    pred = np.dot(w, x)
    # Residual
    residual = pred - y
    # Gradient of loss: d/dw (residual^2) = 2 * residual * x
    grad = 2 * residual * x
    return grad

def grad_loss_batch(w, X_batch, y_batch):
    """
    Compute the gradient of the loss on a mini-batch M:
    ∇ℓ(θ; M) = (1/|M|) Σ_{i∈M} ∇ℓ(θ; x^i, y^i)

    Parameters:
    w: parameter vector, shape (d,)
    X_batch: batch of inputs, shape (d, batch_size)
    y_batch: batch of labels, shape (batch_size,)

    Returns:
    grad: shape (d,), the average gradient of the loss over the mini-batch
    """
    batch_size = X_batch.shape[1]
    # Initialize gradient accumulator
    grad_sum = np.zeros_like(w)
    
    # Compute gradient for each sample and accumulate
    for i in range(batch_size):
        x_i = X_batch[:, i]
        y_i = y_batch[i]
        grad_sum += grad_loss_single(w, x_i, y_i)
    
    # Average over the batch
    grad = grad_sum / batch_size
    return grad


In [8]:
d = 5
batch_size = 10
w = np.random.randn(d)
X_batch = np.random.randn(d, batch_size)
y_batch = np.random.randn(batch_size)

# Compute the gradient of the loss on the batch
gradient = grad_loss_batch(w, X_batch, y_batch)
print("Gradient on the batch:", gradient)

Gradient on the batch: [ 1.69307136  1.01083614  1.16756809 -1.76544991  0.14306195]
